In [2]:
import numpy as np
from ultralytics import YOLO
import cv2
import cvzone
import math
from sort import *
 
cap = cv2.VideoCapture("cars.mp4")  # For Video
 
model = YOLO("yolov8l.pt")
 
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush"
              ]
 
mask = cv2.imread("mask.png")
 
# Tracking
tracker = Sort(max_age=20, min_hits=3, iou_threshold=0.3)
 
limits = [400, 297, 673, 297]
totalCount = []
 
while True:
    success, img = cap.read()
    imgRegion = cv2.bitwise_and(img, mask)
 
    imgGraphics = cv2.imread("graphics.png", cv2.IMREAD_UNCHANGED)
    img = cvzone.overlayPNG(img, imgGraphics, (0, 0))
    results = model(imgRegion, stream=True)
 
    detections = np.empty((0, 5))
 
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # Bounding Box
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3)
            w, h = x2 - x1, y2 - y1
 
            # Confidence
            conf = math.ceil((box.conf[0] * 100)) / 100
            # Class Name
            cls = int(box.cls[0])
            currentClass = classNames[cls]
 
            if currentClass == "car" or currentClass == "truck" or currentClass == "bus" \
                    or currentClass == "motorbike" and conf > 0.3:
                # cvzone.putTextRect(img, f'{currentClass} {conf}', (max(0, x1), max(35, y1)),
                #                    scale=0.6, thickness=1, offset=3)
                # cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=5)
                currentArray = np.array([x1, y1, x2, y2, conf])
                detections = np.vstack((detections, currentArray))
 
    resultsTracker = tracker.update(detections)
 
    cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 0, 255), 5)
    for result in resultsTracker:
        x1, y1, x2, y2, id = result
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
        print(result)
        w, h = x2 - x1, y2 - y1
        cvzone.cornerRect(img, (x1, y1, w, h), l=9, rt=2, colorR=(255, 0, 255))
        cvzone.putTextRect(img, f' {int(id)}', (max(0, x1), max(35, y1)),
                           scale=2, thickness=3, offset=10)
 
        cx, cy = x1 + w // 2, y1 + h // 2
        cv2.circle(img, (cx, cy), 5, (255, 0, 255), cv2.FILLED)
 
        if limits[0] < cx < limits[2] and limits[1] - 15 < cy < limits[1] + 15:
            if totalCount.count(id) == 0:
                totalCount.append(id)
                cv2.line(img, (limits[0], limits[1]), (limits[2], limits[3]), (0, 255, 0), 5)
 
    # cvzone.putTextRect(img, f' Count: {len(totalCount)}', (50, 50))
    cv2.putText(img,str(len(totalCount)),(255,100),cv2.FONT_HERSHEY_PLAIN,5,(50,50,255),8)
 
    cv2.imshow("Image", img)
    # cv2.imshow("ImageRegion", imgRegion)
    cv2.waitKey(1)

Ultralytics YOLOv8.0.26  Python-3.9.0 torch-2.5.1+cpu CPU
YOLOv8l summary (fused): 268 layers, 43668288 parameters, 0 gradients, 165.2 GFLOPs

0: 384x640 3 cars, 991.6ms
Speed: 7.4ms pre-process, 991.6ms inference, 104.2ms postprocess per image at shape (1, 3, 640, 640)



[        571         317         653         430           3]
[        459         225         510         271           2]
[        378         348         496         460           1]


0: 384x640 3 cars, 723.8ms
Speed: 0.0ms pre-process, 723.8ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)



[     569.47      322.76      649.53      436.24           3]
[     458.01      226.99      507.99      272.01           2]
[     373.87      352.13      493.13      465.87           1]


0: 384x640 3 cars, 668.9ms
Speed: 1.3ms pre-process, 668.9ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



...

0: 384x640 2 cars, 1 bus, 1 truck, 671.2ms
Speed: 1.0ms pre-process, 671.2ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



[     621.07      236.97      671.62      286.89          81]
[     589.27       346.9      664.88      422.23          80]
[     228.53      290.47      439.04      486.33          79]
[        380      402.04      497.21      531.26          75]


0: 384x640 2 cars, 1 bus, 1 truck, 620.3ms
Speed: 1.2ms pre-process, 620.3ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)



[     616.66      247.59      671.48      301.92          81]
[     574.14      380.15      652.56      463.88          80]
[     184.29      302.68      418.01      518.56          79]
[     343.73       428.1      474.18       546.2          75]


KeyboardInterrupt: 